In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [1]:
import os
import json
from collections import defaultdict
from pprint import pprint
from feature_extractors import NDVIFeatureExtractor, ColorChannelsFeatureExtractor, LBPFeatureExtractor
from pipeline import complete_pipeline
from sklearn.pipeline import FeatureUnion

In [2]:
import sys
sys.path.append("../")
from config import RESULTS_PATH

# Initialise feature extractors

In [25]:
feature_extractors = [
    ('spectral', FeatureUnion([
        ("channels", ColorChannelsFeatureExtractor()),
        ("ndvi", NDVIFeatureExtractor())
    ]))
]

In [26]:
textural_features = [
    ('lbp_1_8', LBPFeatureExtractor(r=1)),
    ('lbp_2_16', LBPFeatureExtractor(r=2)),
    ('lbp_3_24', LBPFeatureExtractor(r=3))
]
feature_extractors.extend(textural_features)
lbp_multiscale = FeatureUnion(textural_features)
feature_extractors.append(('lbp_multiscale', lbp_multiscale))

In [28]:
feature_extractors

[('spectral', FeatureUnion(n_jobs=None,
         transformer_list=[('channels', <feature_extractors.ColorChannelsFeatureExtractor object at 0x129e45be0>), ('ndvi', <feature_extractors.NDVIFeatureExtractor object at 0x129e45cf8>)],
         transformer_weights=None)),
 ('lbp_1_8', <feature_extractors.LBPFeatureExtractor at 0x129e455c0>),
 ('lbp_2_16', <feature_extractors.LBPFeatureExtractor at 0x129e45e10>),
 ('lbp_3_24', <feature_extractors.LBPFeatureExtractor at 0x129e459b0>),
 ('lbp_multiscale', FeatureUnion(n_jobs=None,
         transformer_list=[('lbp_1_8', <feature_extractors.LBPFeatureExtractor object at 0x129e455c0>), ('lbp_2_16', <feature_extractors.LBPFeatureExtractor object at 0x129e45e10>), ('lbp_3_24', <feature_extractors.LBPFeatureExtractor object at 0x129e459b0>)],
         transformer_weights=None))]

# evaluate performance

In [15]:
performances = defaultdict(dict)
for feature_name, feature_extractor in feature_extractors:
    performance = complete_pipeline(feature_extractor)
    performances[feature_name] = performance

/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average

In [16]:
performances

defaultdict(dict,
            {'spectral': {'mean_f2': 0.8248273151264485,
              'per_class_f2': OrderedDict([('primary', 0.9828969315083),
                           ('clear', 0.9511554803274725),
                           ('agriculture', 0.7320910973084885),
                           ('road', 0.5596966970826373),
                           ('water', 0.5560388574742642),
                           ('partly_cloudy', 0.7164589973920603),
                           ('cultivation', 0.17339544513457558),
                           ('habitation', 0.21659548505186094),
                           ('haze', 0.5135699373695198),
                           ('cloudy', 0.6744186046511629),
                           ('bare_ground', 0.01375515818431912),
                           ('selective_logging', 0.0),
                           ('artisinal_mine', 0.39115646258503406),
                           ('blooming', 0.0),
                           ('slash_burn', 0.0),
                      

In [17]:
for feature_name, performance in performances.items():
    file_path = os.path.join(RESULTS_PATH, f'{feature_name}.json')
    with open(file_path, 'w') as f:
        json.dump(performance, f, indent=4)

In [29]:
from pipeline import extract_features
from helpers.img_utils import load_image_collection
from config import TRAIN_PATH
from time import time

In [22]:
imgs_collection = load_image_collection(TRAIN_PATH)

In [30]:
time_dict = {}
for feature_name, feature_extractor in feature_extractors:
    start = time()
    extract_features(imgs_collection, feature_extractor)
    end = time()
    time_dict[feature_name] = (end - start)

In [31]:
time_dict

{'spectral': 366.9623210430145,
 'lbp_1_8': 771.7961959838867,
 'lbp_2_16': 930.7690010070801,
 'lbp_3_24': 1121.5438289642334,
 'lbp_multiscale': 2650.3443608283997}